In [1]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import psutil

mem_before_load = psutil.virtual_memory()
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

import pyoti.static.onumm1n1    as dual
import pyoti.static.mdnum2      as md2
import pyoti.static.mdnum3      as md3
import pyoti.static.mdnum4      as md4
import pyoti.static.mdnum5      as md5
import pyoti.static.mdnum7      as md7

import pyoti.static.onumm1n2    as om1n2
import pyoti.static.onumm1n3    as om1n3
import pyoti.static.onumm1n4    as om1n4
import pyoti.static.onumm1n5    as om1n5
import pyoti.static.onumm1n10   as om1n10

import pyoti.static.onumm10n1   as om10n1
import pyoti.static.onumm10n2   as om10n2


import pyoti.static.onumm2n1    as om2n1
import pyoti.static.onumm2n2    as om2n2
import pyoti.static.onumm2n3    as om2n3
import pyoti.static.onumm2n4    as om2n4
import pyoti.static.onumm2n5    as om2n5

import pyoti.static.onumm3n1    as om3n1
import pyoti.static.onumm3n2    as om3n2
import pyoti.static.onumm3n3    as om3n3
import pyoti.static.onumm3n4    as om3n4
import pyoti.static.onumm3n5    as om3n5

import pyoti.static.onumm4n1    as om4n1
import pyoti.static.onumm4n2    as om4n2
import pyoti.static.onumm4n3    as om4n3
import pyoti.static.onumm4n4    as om4n4
import pyoti.static.onumm4n5    as om4n5

import pyoti.static.onumm5n1    as om5n1
import pyoti.static.onumm5n2    as om5n2
import pyoti.static.onumm5n3    as om5n3
import pyoti.static.onumm5n4    as om5n4
import pyoti.static.onumm5n5    as om5n5

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')

global times
from timeit import default_timer as time

mem_after_load = psutil.virtual_memory()

In [2]:
def cylinder(ri, ro, he = 1.0, element_order = 1, quads = False, 
           quad_incomplete = 1, quad_linear = 1, structured = False, save=False):
    """
    PORPUSE: Define a cylinder section mesh.
    """
    #***************************************************************************************************
    import gmsh

    gmsh.initialize()
    # gmsh.fltk.initialize()

    # Lets create a simple square element:
    model = gmsh.model
    geo   = model.geo
    option= gmsh.option

    P1 = geo.addPoint( 0.0, 0.0, 0.0, he, 1)
    P2 = geo.addPoint( 0.0,  ri, 0.0, he, 2)
    P3 = geo.addPoint(  ri, 0.0, 0.0, he, 3)
    P4 = geo.addPoint( 0.0,  ro, 0.0, he, 4)
    P5 = geo.addPoint(  ro, 0.0, 0.0, he, 5)

    C1 = geo.addCircleArc(P2,P1,P3)
    C2 = geo.addCircleArc(P5,P1,P4)
    
    geo.remove([P1])
    
    L1 = geo.addLine(P4,P2) # Left
    L2 = geo.addLine(P3,P5) # bottom

    loop1 = geo.addCurveLoop([C1,L2,C2,L1])

    surface = geo.addPlaneSurface([loop1])

    model.addPhysicalGroup( 0, [P1],        100 )
    model.addPhysicalGroup( 0, [P2],        101 )
    model.addPhysicalGroup( 0, [P3],        102 )
    model.addPhysicalGroup( 0, [P4],        103 )
    model.addPhysicalGroup( 0, [P5],        104 )

    model.addPhysicalGroup( 1, [C1],        201 )
    model.addPhysicalGroup( 1, [C2],        202 )
    model.addPhysicalGroup( 1, [L1],        203 )
    model.addPhysicalGroup( 1, [L2],        204 )

    model.addPhysicalGroup( 2, [surface],   301 )
    
    model.setPhysicalName( 0, 100, "center"  )
    model.setPhysicalName( 0, 101, "ri_left"  )
    model.setPhysicalName( 0, 102, "ri_right" )
    model.setPhysicalName( 0, 103, "ro_left"  )
    model.setPhysicalName( 0, 104, "ro_right" )

    model.setPhysicalName( 1, 201, "ri"    )
    model.setPhysicalName( 1, 202, "ro"    )
    model.setPhysicalName( 1, 203, "left"  )
    model.setPhysicalName( 1, 204, "right" )

    model.setPhysicalName( 2, 301, "domain"      )

    
    geo.synchronize()

    option.setNumber('Mesh.ElementOrder',element_order)


    # Recombine if quads are wanted.
    if quads:

        # Set body to recombine into quads.
        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         1)

    else:

        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         0)

    # end if 

    if structured:
        nels_L = int((ro-ri)/he)+1
        nels_C = int((ri*np.pi*2)/(4*he))+1
        model.mesh.setTransfiniteCurve(L1,nels_L)
        model.mesh.setTransfiniteCurve(L2,nels_L)
        model.mesh.setTransfiniteCurve(C1,nels_C)
        model.mesh.setTransfiniteCurve(C2,nels_C)
        model.mesh.setTransfiniteSurface(surface)#,"left",[P2,P3,P4])
    # end if 

    model.mesh.generate(2)

    if save:
        gmsh.write("cylinder.msh")
    # end if 

    Th = fem.mesh.from_gmsh(gmsh)
#     Th = None

    gmsh.finalize()

    return Th

#-----------------------------------------------------------------------------------------------------

In [3]:
#*****************************************************************************************************
def solve_2d_linear_elasticity(Th, E, nu, ri, Pi, ro, Po, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    mu = E/(2*(1+nu))
    
    c1 = E*nu/((1+nu)*(1-2*nu))
    c2 = 2*mu
    c3 = mu
    c4 = c1+c2
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    nNodes = Th.x.shape[0]
    nDOF = 2*nNodes
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    K[nNodes,nNodes] = 1.0 # Removes "center" node.
    K[0,0] = 1.0 # Removes "center" node.
    
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
            # Temps
            
            NxT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            NyT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            
            NxNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NxNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )

            NN_tmp1 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp2 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp3 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            
            tmp11 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp12 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp21 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp22 = alg.zeros( ( elem.nbasis, elem.nbasis ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            alg.transpose(Nx, out = NxT)
            alg.transpose(Ny, out = NyT)
            
            alg.dot( NxT, Nx, out = NxNx )
            alg.dot( NyT, Ny, out = NyNy )
            alg.dot( NyT, Nx, out = NyNx )
            alg.dot( NxT, Ny, out = NxNy )

            # problem = intV( c1*dx(vx)*dx(ux) + c1*dy(vy)*dx(ux) ) + \
            #           intV( c1*dx(vx)*dy(uy) + c1*dy(vy)*dy(uy) ) + \
            #           intV( c2*dx(vx)*dx(ux) + c2*dy(vy)*dy(uy) ) + \
            #           intV( c3*dy(vx)*dy(ux) + c3*dx(vy)*dy(ux) ) + \
            #           intV( c3*dy(vx)*dx(uy) + c3*dx(vy)*dx(uy) ) + \
            #           intS( 'in' , fxi*vx + fyi*vy ) + \
            #           intS( 'out', fxo*vx + fyo*vy ) + \
            #           on(   'left' , ux, 0.0) + \
            #           on(   'right', uy, 0.0)
            #  
            # c1*NxNx + c2*NxNx + c3*NyNy # vx, ux
            # c1*NxNy + c3*NyNx           # vx, uy
            # c1*NyNx + c3*NxNy           # vy, ux
            # c1*NyNy + c2*NyNy + c3*NxNx # vy, uy
            
            # c4 = c1+c2

#             tmp11.set(0)
#             tmp12.set(0)
#             tmp21.set(0)
#             tmp22.set(0)
            
            # alg.gauss_integrate( c4*NxNx + c3*NyNy , elem.dV,out=tmp11) # vx,ux            
            alg.mul(      c4,    NxNx, out=NN_tmp1 )
            alg.mul(      c3,    NyNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp11 ) # vx,ux
            
            # alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV,out=tmp12) # vx,uy
            alg.mul(      c1,    NxNy, out=NN_tmp1 )
            alg.mul(      c3,    NyNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp12 ) # vx,uy
            
            # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vy,ux
            alg.mul(      c1,    NyNx, out=NN_tmp1 )
            alg.mul(      c3,    NxNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp21 ) # vy,ux
            
            # alg.gauss_integrate( c4*NyNy + c3*NxNx , elem.dV,out=tmp22) # vy,uy
            alg.mul(      c4,    NyNy, out=NN_tmp1 )
            alg.mul(      c3,    NxNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp22 ) # vy,uy
            

                      
            # assemble globals
            
            for k in range(elems.size):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii + nNodes # Move DOF for second variable
                
                for l in range(elems.size):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nNodes # Move DOF for second variable
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]
                    
                    K[i1,j2] = K[i1,j2] + tmp12[k,l]
                    
                    K[i2,j1] = K[i2,j1] + tmp21[k,l]
                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   

    end_assmbly_time = time()
    
    
    # 
    els = Th.group_names['ri']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            nxel = alg.zeros( ( elem.nbasis, 1 ) )
            xel  = alg.zeros( ( elem.nbasis, 1 ) )
            nyel = alg.zeros( ( elem.nbasis, 1 ) )
            yel  = alg.zeros( ( elem.nbasis, 1 ) )
            

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)  
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ri
            nyel = yel/ri
            
            
            tmp1 = alg.gauss_integrate( (Pi)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (Pi)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            
#             print("tmp1[0,0].order",tmp1[0,0].order)
#             print("tmp2[0,0].order",tmp2[0,0].order)
            
            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for
#     print("f.order", f.order)

    fem.end_elements()  
    
    els = Th.group_names['ro']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            fh = alg.zeros( ( elem.nbasis, 1 ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ro
            nyel = yel/ro
            
            
            tmp1 = alg.gauss_integrate( (-Po)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (-Po)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            

            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for
#     print("f.order", f.order)

    fem.end_elements()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_) # uy
            i2 = int(ii_)+nNodes # uy

            K[i2,i2] = TGV
            f[i2,0] = 0.0
            
#             K[i1,i1] = TGV
#             f[i1,0] = 0.1*TGV
        
        # end for 
        
        
    # end for
    
    els = Th.group_names['left']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_) # ux
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for
    end_bc_time = time()
    
#     print("f.order", f.order)
    K = K.tocsr()
    u = alg.solve( K, f )
    # u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 

    return u,K,f

    #-----------------------------------------------------------------------------------------------------

In [4]:
#*****************************************************************************************************
def solve_2d_linear_elasticity_bruteforce(Th, E, nu, ri, Pi, ro, Po, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    mu = E/(2*(1+nu))
    
    c1 = E*nu/((1+nu)*(1-2*nu))
    c2 = 2*mu
    c3 = mu
    c4 = c1 + c2
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    nNodes = Th.x.shape[0]
    nDOF = 2*nNodes
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    K[nNodes,nNodes] = 1.0 # Removes "center" node.
    K[0,0] = 1.0 # Removes "center" node.
    
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
            # Temps
            
#             NxT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
#             NyT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            NxNx = alg.dot(alg.transpose(Nx),Nx)
            NyNy = alg.dot(alg.transpose(Ny),Ny)
            NyNx = alg.dot(alg.transpose(Ny),Nx)
            NxNy = alg.dot(alg.transpose(Nx),Ny)

#             problem = intV( c1*dx(vx)*dx(ux) + c1*dy(vy)*dx(ux) ) + \
#                       intV( c1*dx(vx)*dy(uy) + c1*dy(vy)*dy(uy) ) + \
#                       intV( c2*dx(vx)*dx(ux) + c2*dy(vy)*dy(uy) ) + \
#                       intV( c3*dy(vx)*dy(ux) + c3*dx(vy)*dy(ux) ) + \
#                       intV( c3*dy(vx)*dx(uy) + c3*dx(vy)*dx(uy) ) + \
#                       intS( 'in' , fxi*vx + fyi*vy ) + \
#                       intS( 'out', fxo*vx + fyo*vy ) + \
#                       on(   'left' , ux, 0.0) + \
#                       on(   'right', uy, 0.0)

#             c1*NxNx + c2*NxNx + c3*NyNy # vx, ux
#             c1*NxNy + c3*NyNx           # vx, uy
#             c1*NyNx + c3*NxNy           # vy, ux
#             c1*NyNy + c2*NyNy + c3*NxNx # vy, uy
            
            tmp11 = alg.gauss_integrate( c4*NxNx + c3*NyNy , elem.dV) # vx,ux
            tmp12 = alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV) # vx,uy
            tmp21 = alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV) # vy,ux
            tmp22 = alg.gauss_integrate( c4*NyNy + c3*NxNx , elem.dV) # vy,uy
            
                      
            # assemble globals
            
            for k in range(elems.size):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii + nNodes # Move DOF for second variable
                
                for l in range(elems.size):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nNodes # Move DOF for second variable
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]
                    
                    K[i1,j2] = K[i1,j2] + tmp12[k,l]
                    
                    K[i2,j1] = K[i2,j1] + tmp21[k,l]
                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   
    
    
    
    end_assmbly_time = time()
    
    
    # 
    els = Th.group_names['ri']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            nxel = alg.zeros( ( elem.nbasis, 1 ) )
            xel  = alg.zeros( ( elem.nbasis, 1 ) )
            nyel = alg.zeros( ( elem.nbasis, 1 ) )
            yel  = alg.zeros( ( elem.nbasis, 1 ) )
            

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)  
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ri
            nyel = yel/ri
            
            
            tmp1 = alg.gauss_integrate( (Pi)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (Pi)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            
            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()  
    
    els = Th.group_names['ro']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            fh = alg.zeros( ( elem.nbasis, 1 ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ro
            nyel = yel/ro
            
            
            tmp1 = alg.gauss_integrate( (-Po)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (-Po)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            

            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_) # uy
            i2 = int(ii_)+nNodes # uy

            K[i2,i2] = TGV
            f[i2,0] = 0.0
            
#             K[i1,i1] = TGV
#             f[i1,0] = 0.1*TGV
        
        # end for 
        
        
    # end for
    
    els = Th.group_names['left']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_) # ux
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for
    end_bc_time = time()
    
   
    u = alg.solve(K.tocsr(),f)
#     u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 

    return u,K,f

    #-----------------------------------------------------------------------------------------------------
    
def assemble_globals(Kel,fel,Kgl,fgl,elem):
    
    
    for i in range(elem.size):
        
        ii=int(elem[i])
        
        for j in range(elem.size):

            jj=int(elem[j])
        
            Kgl[ii,jj] = Kgl[ii,jj] + Kel[i,j]
            
        # end for 
        fgl[ii,0] = fgl[ii,0] + fel[i,0]
        
    # end for 

    
def analytic_solution(Th,E,nu,ri,pi,ro,po):
    
    A = ri**2*ro**2*(po-pi)/(ro**2-ri**2)
    B = (ri**2*pi - ro**2*po)/(ro**2-ri**2)
    
    x = Th.x
    y = Th.y
    
    r =  (x**2+y**2)**0.5
    
    ur = (1+nu)*r/E*((1-2*nu)*B-A/(r**2))
    
    ux = ur*x/r
    uy = ur*y/r
    
    return ux, uy

In [5]:
# r,  oti, 
# dual, md2, md3, md4, md5, md7
# om1n2, om1n3, om1n4, om1n5, om1n10
# om10n1, om10n2
# om2n1, om2n2, om2n3, om2n4, om2n5
# om3n3, om3n4, om3n5
# om4n4, om4n5
# om5n5, om5n5


fem.set_global_algebra(oti)
alg = fem.get_global_algebra()





start_time = time()


Th = cylinder(1,2,he=0.04,quads=False,save=True, structured=True)   # 
# Th = cylinder(1,2,he=0.0027,quads=False,save=True, structured=True)  # 500K DOFs 
# Th = cylinder(1,2,he=0.00177,quads=False,save=True, structured=True) # 1M DOFs
# Th = cylinder(1,2,he=0.01,quads=False,save=True, structured=True)



end_time = time()

print(end_time - start_time,"seg")


mem_after_mesh = psutil.virtual_memory()
Th

0.14085011999122798 seg


/home/maristi7/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


< mesh (pyoti.sparse) object with 1041 nodes, 2083 elements of types ( point1 (5), line2 (128), tri3 (1950) ) >

In [8]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []

E  = alg.number(21e5)+alg.e(1,order=1)#,order=1)#,1,10,2,1000
nu = alg.number(0.28)+alg.e(2,order=10)
ri = alg.number(1)
Pi = alg.number(10)  +alg.e(3,order=1)# Pi = alg.number(1400)
ro = alg.number(2)
Po = alg.number(1000)+alg.e(4,order=1)
mem_before_FEM = psutil.virtual_memory()
for i in range(1):
#     u_res,K,f = solve_2d_linear_elasticity(Th,E,nu,ri,Pi,ro,Po)
    u_res,K,f = solve_2d_linear_elasticity(Th,E,nu,ri,Pi,ro,Po)
#end for
mem_after_FEM = psutil.virtual_memory()
print('\n\nTimes:'+str(type(u_res)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )
print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )

Total run time: 3.603116 s 



Times:<class 'pyoti.sparse.matso'>
- Avg Assembly time:  0.453090 s 
- Avg Boundary time:  0.001917 s 
- Avg Solution time:  3.148109 s 
- Avg Total run time: 3.603116 s 


In [9]:
u = alg.zeros((u_res.shape[0]//2,3))
u[:,0:1] = u_res[ :u_res.shape[0]//2, : ]
u[:,1:2] = u_res[ u_res.shape[0]//2:, : ]

u[0,0:2] = np.nan
u.real

u_r = np.linalg.norm(u.real,axis=1).reshape((u.shape[0],-1))

In [10]:
# XPS (1000536) triangles (501721 x 2) DOF

# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  24.585664 s 
# - Avg Boundary time:  0.014931 s 
# - Avg Solution time:  68.401785 s 
# - Avg Total run time: 93.002380 s 

# Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
# - Avg Assembly time:  42.544440 s 
# - Avg Boundary time:  0.029241 s 
# - Avg Solution time:  73.244128 s 
# - Avg Total run time: 115.817809 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  45.466606 s 
# - Avg Boundary time:  0.033116 s 
# - Avg Solution time:  77.474226 s 
# - Avg Total run time: 122.973948 s 

# Times:<class 'pyoti.static.mdnum3.mdmat3'>
# - Avg Assembly time:  48.692000 s 
# - Avg Boundary time:  0.036328 s 
# - Avg Solution time:  85.212966 s 
# - Avg Total run time: 133.941294 s 

# Times:<class 'pyoti.static.mdnum4.mdmat4'>
# - Avg Assembly time:  62.095158 s 
# - Avg Boundary time:  0.040278 s 
# - Avg Solution time:  96.761277 s 
# - Avg Total run time: 158.896713 s 

# Times:<class 'pyoti.static.mdnum5.mdmat5'>
# - Avg Assembly time:  109.387500 s 
# - Avg Boundary time:  0.054357 s 
# - Avg Solution time:  117.766977 s 
# - Avg Total run time: 227.208834 s 


# Times:<class 'pyoti.static.onumm10n1.omatm10n1'>
# - Avg Assembly time:  47.717444 s 
# - Avg Boundary time:  0.031522 s 
# - Avg Solution time:  80.154365 s 
# - Avg Total run time: 127.903331 s 

# Times:<class 'pyoti.static.onumm10n2.omatm10n2'>
# - Avg Assembly time:  119.280657 s 
# - Avg Boundary time:  0.061506 s 
# - Avg Solution time:  126.035270 s 
# - Avg Total run time: 245.377434 s 


# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  43.654764 s 
# - Avg Boundary time:  0.033134 s 
# - Avg Solution time:  76.820544 s 
# - Avg Total run time: 120.508442 s 

# Times:<class 'pyoti.static.onumm1n3.omatm1n3'>
# - Avg Assembly time:  45.273250 s 
# - Avg Boundary time:  0.035113 s 
# - Avg Solution time:  81.524970 s 
# - Avg Total run time: 126.833334 s 

# Times:<class 'pyoti.static.onumm1n4.omatm1n4'>
# - Avg Assembly time:  46.402611 s 
# - Avg Boundary time:  0.037255 s 
# - Avg Solution time:  87.030372 s 
# - Avg Total run time: 133.470238 s

# Times:<class 'pyoti.static.onumm1n5.omatm1n5'>
# - Avg Assembly time:  48.011230 s 
# - Avg Boundary time:  0.039146 s 
# - Avg Solution time:  92.742498 s 
# - Avg Total run time: 140.792874 s 




# Times:<class 'pyoti.static.onumm2n1.omatm2n1'>
# - Avg Assembly time:  42.165037 s 
# - Avg Boundary time:  0.030219 s 
# - Avg Solution time:  74.535828 s 
# - Avg Total run time: 116.731085 s 

# Times:<class 'pyoti.static.onumm2n2.omatm2n2'>
# - Avg Assembly time:  47.393488 s 
# - Avg Boundary time:  0.033968 s 
# - Avg Solution time:  79.338687 s 
# - Avg Total run time: 126.766142 s 

# Times:<class 'pyoti.static.onumm2n3.omatm2n3'>
# - Avg Assembly time:  50.611038 s 
# - Avg Boundary time:  0.036769 s 
# - Avg Solution time:  86.186553 s 
# - Avg Total run time: 136.834359 s 

# Times:<class 'pyoti.static.onumm2n4.omatm2n4'>
# - Avg Assembly time:  60.127457 s 
# - Avg Boundary time:  0.041286 s 
# - Avg Solution time:  96.422317 s 
# - Avg Total run time: 156.591060 s 

# Times:<class 'pyoti.static.onumm2n5.omatm2n5'>
# - Avg Assembly time:  75.727625 s 
# - Avg Boundary time:  0.046857 s 
# - Avg Solution time:  108.150166 s 
# - Avg Total run time: 183.924648 s 


# Times:<class 'pyoti.static.onumm3n1.omatm3n1'>
# - Avg Assembly time:  42.932034 s 
# - Avg Boundary time:  0.030055 s 
# - Avg Solution time:  75.749584 s 
# - Avg Total run time: 118.711673 s 

# Times:<class 'pyoti.static.onumm3n2.omatm3n2'>
# - Avg Assembly time:  47.224021 s 
# - Avg Boundary time:  0.034291 s 
# - Avg Solution time:  83.086619 s 
# - Avg Total run time: 130.344930 s 
    
# Times:<class 'pyoti.static.onumm3n3.omatm3n3'>
# - Avg Assembly time:  61.927626 s 
# - Avg Boundary time:  0.040087 s 
# - Avg Solution time:  94.717598 s 
# - Avg Total run time: 156.685310 s 

# Times:<class 'pyoti.static.onumm3n4.omatm3n4'>
# - Avg Assembly time:  97.031870 s 
# - Avg Boundary time:  0.053130 s 
# - Avg Solution time:  114.032486 s 
# - Avg Total run time: 211.117487 s 


# Times:<class 'pyoti.static.onumm3n5.omatm3n5'>
# - Avg Assembly time:  166.864188 s 
# - Avg Boundary time:  0.072990 s 
# - Avg Solution time:  143.523192 s 
# - Avg Total run time: 310.460370 s 
    




# Times:<class 'pyoti.static.onumm4n1.omatm4n1'>
# - Avg Assembly time:  42.856982 s 
# - Avg Boundary time:  0.030389 s 
# - Avg Solution time:  77.197110 s 
# - Avg Total run time: 120.084481 s 

# Times:<class 'pyoti.static.onumm4n2.omatm4n2'>
# - Avg Assembly time:  53.402267 s 
# - Avg Boundary time:  0.036895 s 
# - Avg Solution time:  87.502197 s 
# - Avg Total run time: 140.941358 s 

# Times:<class 'pyoti.static.onumm4n3.omatm4n3'>
# - Avg Assembly time:  89.312343 s 
# - Avg Boundary time:  0.047902 s 
# - Avg Solution time:  106.748769 s 
# - Avg Total run time: 196.109015 s 

# Times:<class 'pyoti.static.onumm4n4.omatm4n4'>
# - Avg Assembly time:  179.295920 s 
# - Avg Boundary time:  0.082577 s 
# - Avg Solution time:  141.456515 s 
# - Avg Total run time: 320.835012 s 


# Times:<class 'pyoti.static.onumm4n5.omatm4n5'>
# () HITS MEMORY BLOCK....




# Times:<class 'pyoti.static.onumm5n1.omatm5n1'>
# - Avg Assembly time:  43.707132 s 
# - Avg Boundary time:  0.030649 s 
# - Avg Solution time:  78.091933 s 
# - Avg Total run time: 121.829714 s 

# Times:<class 'pyoti.static.onumm5n2.omatm5n2'>
# - Avg Assembly time:  59.509572 s 
# - Avg Boundary time:  0.038993 s 
# - Avg Solution time:  92.308948 s 
# - Avg Total run time: 151.857514 s

# Times:<class 'pyoti.static.onumm5n3.omatm5n3'>
# - Avg Assembly time:  126.651041 s 
# - Avg Boundary time:  0.054509 s 
# - Avg Solution time:  124.634182 s 
# - Avg Total run time: 251.339732 s 

# Times:<class 'pyoti.static.onumm5n4.omatm5n4'>
# () HITS MEMORY BLOCK....

# Times:<class 'pyoti.static.onumm5n5.omatm5n5'>
# () HITS MEMORY BLOCK....

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  62.967360 s 
# - Avg Boundary time:  0.050608 s 
# - Avg Solution time:  142.152283 s 
# - Avg Total run time: 205.170251 s 

In [11]:
# XPS  (429940) Triangles (215923 x 2) DOF

# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  11.113099 s 
# - Avg Boundary time:  0.009709 s 
# - Avg Solution time:  17.195912 s 
# - Avg Total run time: 28.318721 s 

# Times:<class 'pyoti.static.onumm1n1.omatm1n1'> (Duals)
# - Avg Assembly time:  17.557875 s 
# - Avg Boundary time:  0.018980 s 
# - Avg Solution time:  18.858839 s 
# - Avg Total run time: 36.435693 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  20.367134 s 
# - Avg Boundary time:  0.030285 s 
# - Avg Solution time:  20.444894 s 
# - Avg Total run time: 40.842314 s 

# Times:<class 'pyoti.static.mdnum3.mdmat3'>
# - Avg Assembly time:  21.308258 s 
# - Avg Boundary time:  0.022566 s 
# - Avg Solution time:  23.918862 s 
# - Avg Total run time: 45.249685 s



# Times:<class 'pyoti.static.mdnum5.mdmat5'>
# - Avg Assembly time:  47.205461 s 
# - Avg Boundary time:  0.046123 s 
# - Avg Solution time:  36.910316 s 
# - Avg Total run time: 84.161900 s 







# Times:<class 'pyoti.static.onumm1n5.omatm1n5'>
# - Avg Assembly time:  21.202655 s 
# - Avg Boundary time:  0.030977 s 
# - Avg Solution time:  26.509539 s 
# - Avg Total run time: 47.743171 s 

# Times:<class 'pyoti.static.onumm2n5.omatm2n5'>
# - Avg Assembly time:  31.470576 s 
# - Avg Boundary time:  0.028044 s 
# - Avg Solution time:  31.799710 s 
# - Avg Total run time: 63.298330 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'> (10 derivs)
# - Avg Assembly time:  26.752567 s 
# - Avg Boundary time:  0.029071 s 
# - Avg Solution time:  48.612258 s 
# - Avg Total run time: 75.393896 s 





# Times:<class 'pyoti.sparse.matso'>  (No derivs)
# - Avg Assembly time:  35.950992 s 
# - Avg Boundary time:  0.023023 s 
# - Avg Solution time:  18.104725 s 
# - Avg Total run time: 54.078741 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 1) (4 derivs)
# - Avg Assembly time:  43.191356 s 
# - Avg Boundary time:  0.024698 s 
# - Avg Solution time:  22.919118 s 
# - Avg Total run time: 66.135172 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 2) (14 derivs)
# - Avg Assembly time:  48.721740 s 
# - Avg Boundary time:  0.025488 s 
# - Avg Solution time:  32.897843 s 
# - Avg Total run time: 81.645071 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 3) (34 derivs)
# - Avg Assembly time:  53.962597 s 
# - Avg Boundary time:  0.027144 s 
# - Avg Solution time:  51.565640 s 
# - Avg Total run time: 105.555382 s

# Times:<class 'pyoti.sparse.matso'> (4 var order 4) (69 derivs)
# - Avg Assembly time:  60.701763 s 
# - Avg Boundary time:  0.027271 s 
# - Avg Solution time:  79.216957 s 
# - Avg Total run time: 139.945991 s 


# Times:<class 'pyoti.sparse.matso'> (4 var order 5)  (126 derivs)
# - Avg Assembly time:  65.937784 s 
# - Avg Boundary time:  0.028036 s 
# - Avg Solution time:  120.965043 s 
# - Avg Total run time: 186.930863 s 

# Times:<class 'pyoti.sparse.matso'> (1 var order 10) (10 derivs - E)
# - Avg Assembly time:  74.959733 s 
# - Avg Boundary time:  0.023045 s 
# - Avg Solution time:  459.551980 s 
# - Avg Total run time: 534.534759 s 

# Times:<class 'pyoti.sparse.matso'> (1 var order 10) (10 derivs - Pi)
# - Avg Assembly time:  36.388555 s 
# - Avg Boundary time:  0.026632 s 
# - Avg Solution time:  17.867372 s 
# - Avg Total run time: 54.282560 s 

In [24]:
deriv = [[2,2]]
dudE = u.get_deriv(deriv)
dudE[0,0:2] = np.nan
dudE_r = np.linalg.norm(dudE.real,axis=1).reshape((dudE.shape[0],-1))
dudE

array([[            nan,             nan,  0.00000000e+00],
       [-1.91912625e-29,  2.48081362e-03,  0.00000000e+00],
       [ 2.63617213e-03, -5.47091566e-29,  0.00000000e+00],
       ...,
       [ 4.77380932e-03,  1.94612871e-04,  0.00000000e+00],
       [ 4.87424763e-03,  1.98702834e-04,  0.00000000e+00],
       [ 4.97487022e-03,  2.02637247e-04,  0.00000000e+00]])

In [25]:
nu.order

10

In [26]:
ux_a, uy_a = analytic_solution(Th,E,nu,ri,Pi,ro,Po)
u_a = alg.zeros(u.shape)
u_a[:,0:1]=ux_a
u_a[:,1:2]=uy_a
u_a.real
u_ra = np.linalg.norm(u_a.real,axis=1).reshape((u_a.shape[0],-1))
u_ra

array([[       nan],
       [0.00116126],
       [0.00116126],
       ...,
       [0.00109855],
       [0.0011039 ],
       [0.00110961]])

In [27]:
u.get_deriv([[2,2]])

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.91912625e-29,  2.48081362e-03,  0.00000000e+00],
       [ 2.63617213e-03, -5.47091566e-29,  0.00000000e+00],
       ...,
       [ 4.77380932e-03,  1.94612871e-04,  0.00000000e+00],
       [ 4.87424763e-03,  1.98702834e-04,  0.00000000e+00],
       [ 4.97487022e-03,  2.02637247e-04,  0.00000000e+00]])

In [28]:
u_a.get_deriv([[2,2]])

array([[0.        , 0.        , 0.        ],
       [0.        , 0.00253333, 0.        ],
       [0.00253333, 0.        , 0.        ],
       ...,
       [0.0047588 , 0.00019177, 0.        ],
       [0.00486006, 0.00019585, 0.        ],
       [0.00496131, 0.00019993, 0.        ]])

In [29]:
u_a.get_deriv([[2,10]])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [30]:
dudE_a = u_a.get_deriv(deriv)
dudE_a[0,0:2] = np.nan
dudE_ra = np.linalg.norm(dudE_a.real,axis=1).reshape((dudE_a.shape[0],-1))
dudE_a

array([[       nan,        nan, 0.        ],
       [0.        , 0.00253333, 0.        ],
       [0.00253333, 0.        , 0.        ],
       ...,
       [0.0047588 , 0.00019177, 0.        ],
       [0.00486006, 0.00019585, 0.        ],
       [0.00496131, 0.00019993, 0.        ]])

In [31]:
eps = 1e-120
u_rerr   =np.abs(u_r-u_ra)/u_ra
dudE_rerr=np.abs(dudE_r - dudE_ra)/(dudE_ra)


In [32]:
u_rerr

array([[       nan],
       [0.00121161],
       [0.00319746],
       ...,
       [0.00077726],
       [0.00074565],
       [0.00072008]])

In [33]:
dudE_rerr

array([[       nan],
       [0.02073147],
       [0.04059426],
       ...,
       [0.00317215],
       [0.00293916],
       [0.0027515 ]])

In [34]:
ux_a, uy_a = analytic_solution(Th,21e5,0.28,1,10,2,1000)
u_a = np.zeros_like(u)
u_a[:,0:1]=ux_a.real
u_a[:,1:2]=uy_a.real
u_a

IndexError: too many indices for array

In [38]:
import pyvista as pv
p = pv.BackgroundPlotter()
args_cbar = dict(height=0.75, vertical=True, position_x=0.05, 
                 position_y=0.05, interactive=False,
                 title_font_size=30, label_font_size=30)

In [39]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u.real],pd_names=['u'])
factor = 100
p.clear()

arrows = grid.glyph(scale="u", orient="u", factor = 100,tolerance=0.001)

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()


In [37]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u_r.real],pd_names=['u'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

NameError: name 'p' is not defined

In [37]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_ra.real],pd_names=['u_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

	`.extract_geometry().tri_filter()`


In [38]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_rerr],pd_names=['relative error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [30]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [31]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [33]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_rerr],pd_names=['relative error (deriv)'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [32]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_rerr],pd_names=['Error dudE_oti'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

AttributeError: BackgroundPlotter has no attribute named ren_win

In [29]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_a],pd_names=['dudE_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [36]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE-dudE_a],pd_names=['10th deriv  abs. error.'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [22]:
x = Th.x.real
y = Th.y.real

In [26]:
# Analytic solution
def analytic_solution(x,y,E,nu,ri,pi,ro,po):
    
    A = ri**2*ro**2*(po-pi)/(ro**2-ri**2)
    B = (ri**2*pi - ro**2*po)/(ro**2-ri**2)

    
    r =  (x**2+y**2)**0.5
    
    ur = (1+nu)*r/E*((1-2*nu)*B-A/(r**2))
    
    ux = ur*x/r
    uy = ur*y/r
    
    return ux, uy

In [29]:
import sympy as sym
Es, nus, ris, Pis, ros, Pos, xs, ys = sym.symbols("E nu r_i P_i r_o P_o x y")

In [30]:
uxs, uys = analytic_solution(xs,ys,Es,nus,ris,Pis,ros,Pos)

In [35]:
ux_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uxs)
duxdE_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uxs.diff(Es,1))

uy_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uys)
duydE_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uys.diff(Es,1))

In [36]:
ux_eval(1,0,E.real,nu.real,ri.real, Pi.real, ro.real, Po.real)

-0.001161264761904762

In [37]:
duxdE_eval(1,0,E.real,nu.real,ri.real, Pi.real, ro.real, Po.real)

5.529832199546485e-10

In [2]:
f = oti.zeros((10,1))

In [3]:
f

matso< shape: (10, 1), 
 - Column 0
(0,0) 0
(1,0) 0
(2,0) 0
(3,0) 0
(4,0) 0
(5,0) 0
(6,0) 0
(7,0) 0
(8,0) 0
(9,0) 0
>

In [8]:
f[5,0]=3.5+oti.e(10,order=10)

In [9]:
f

matso< shape: (10, 1), 
 - Column 0
(0,0) 0
(1,0) 0
(2,0) 0
(3,0) 0
(4,0) 0
(5,0) 3.5 + 1 * e([10])
(6,0) 0
(7,0) 0
(8,0) 0
(9,0) 0
>

In [10]:
f.order

0

In [12]:
f[5,0].order

10